In [92]:
import pandas as pd
import numpy as np
import string
import time
import random
import unidecode
import math
import operator
import os

import torch
import torch.nn as nn
from torch.autograd import Variable

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

In [4]:
device = torch.device("cpu")

In [147]:
class Dictionary(object):
    def __init__(self):
        self.count_threshold = 60
        self.occurrences = {}
        self.word2idx = {}
        self.idx2word = []
        
    def cleanup(self):
        new_word2idx = {}
        new_idx2word = []
        for w in self.occurrences:
            if self.occurrences[w] < self.count_threshold:
                continue
            else:
                new_idx2word.append(w)
                new_word2idx[w] = len(new_idx2word)
        self.word2idx = new_word2idx
        self.idx2word = new_idx2word

    def add_word(self, word):
        if word not in self.word2idx:
            self.idx2word.append(word)
            self.word2idx[word] = len(self.idx2word) - 1
            self.occurrences[word] = 1
        else: self.occurrences[word] += 1
        return self.word2idx[word]

    def __len__(self):
        return len(self.idx2word)

class Corpus(object):
    def __init__(self, train, valid, test):
        self.dictionary = Dictionary()
        self.train = self.tokenize(train)
        self.valid = self.tokenize(valid)
        self.test = self.tokenize(test)

    def tokenize(self, path):
        # Add words to the dictionary
        tokens = 0
        for line in path:
            words = word_tokenize(line) + ['\n']
            tokens += len(words)
            for word in words:
                self.dictionary.add_word(word)
                
        self.dictionary.cleanup()

        # Tokenize file content
        ids = torch.LongTensor(tokens)
        token = 0
        for line in path:
            words = word_tokenize(line) + ['\n']
            for word in words:
                if word not in self.dictionary.word2idx: continue
                ids[token] = self.dictionary.word2idx[word]
                token += 1

        return ids

In [148]:
class Dictionary2(object):
    def __init__(self):
        self.word2idx = {}
        self.idx2word = []

    def add_word(self, word):
        if word not in self.word2idx:
            self.idx2word.append(word)
            self.word2idx[word] = len(self.idx2word) - 1
        return self.word2idx[word]

    def __len__(self):
        return len(self.idx2word)


class Corpus2(object):
    def __init__(self, path):
        self.dictionary = Dictionary2()
        self.train = self.tokenize(os.path.join(path, 'train.txt'))
        self.valid = self.tokenize(os.path.join(path, 'valid.txt'))
        self.test = self.tokenize(os.path.join(path, 'test.txt'))

    def tokenize(self, path):
        """Tokenizes a text file."""
        assert os.path.exists(path)
        # Add words to the dictionary
        with open(path, 'r') as f:
            tokens = 0
            for line in f:
                words = line.split() + ['\n']
                tokens += len(words)
                for word in words:
                    self.dictionary.add_word(word)

        # Tokenize file content
        with open(path, 'r') as f:
            ids = torch.LongTensor(tokens)
            token = 0
            for line in f:
                words = line.split() + ['\n']
                for word in words:
                    ids[token] = self.dictionary.word2idx[word]
                    token += 1

            return ids

In [149]:
lines = pd.read_csv("the-office-lines-simplified.csv")
characters = {
    "Michael":1,"Dwight":2,"Jim":3,"Pam":4,"Andy":5,"Kevin":6,"Angela":7,
    "Oscar":8,"Erin":9,"Ryan":10,"Darryl":11,"Phyllis":12,"Kelly":13,"Jan":14,"Toby":15,"Other":16
}

characters_inverted = {}
for c in characters:
    characters_inverted[characters[c]] = c

full_text = ""
for i, line in enumerate(lines['line_text']):
    full_text += characters_inverted[lines['speaker_id'][i]] + ': ' + lines['line_text'][i].lower() + '\n'
    
all_lines = full_text.split('\n')
t1 = int(round(8*len(all_lines)/10.0))
t2 = int(round(9*len(all_lines)/10.0))

outtrain = open("./data/office/train.txt", 'w+')
outvalid = open("./data/office/valid.txt", 'w+')
outtest = open("./data/office/test.txt", 'w+')

for line in all_lines[:t1]:
    outtrain.write(line + '\n')
for line in all_lines[t1:t2]:
    outvalid.write(line + '\n')  
for line in all_lines[t2:]:
    outtest.write(line + '\n')  

#p1 = open("/home/rana/tmp/examples/word_language_model/data/wikitext-2/train.txt", 'r')
#p2 = open("~\tmp\examples\word-language-model\data\wikitext-2\train.txt", 'r')
#p3 = open("~\tmp\examples\word-language-model\data\wikitext-2\train.txt", 'r')
#c = Corpus(all_lines[:t1], all_lines[t1:t2], all_lines[t2:])
#c = Corpus2("/home/rana/tmp/examples/word_language_model/data/wikitext-2")
c = Corpus2("./data/office")

In [108]:
c.dictionary.word2idx

{'!': 563,
 '$': 307,
 "'": 51,
 "'cause": 87,
 "'d": 434,
 "'em": 283,
 "'ll": 836,
 "'m": 435,
 "'re": 573,
 "'s": 433,
 "'ve": 136,
 ',': 746,
 '-': 384,
 '--': 507,
 '.': 9,
 '...': 308,
 ':': 681,
 ';': 313,
 '<eos>': 79,
 '?': 524,
 'Andy': 680,
 'Angela': 206,
 'Darryl': 652,
 'Dwight': 158,
 'Erin': 76,
 'Jan': 70,
 'Jim': 44,
 'Kelly': 141,
 'Kevin': 653,
 'Michael': 454,
 'Oscar': 12,
 'Other': 101,
 'Pam': 786,
 'Phyllis': 90,
 'Ryan': 687,
 'Toby': 230,
 '[': 393,
 ']': 499,
 'a': 720,
 'able': 738,
 'about': 578,
 'absolutely': 146,
 'accent': 828,
 'actually': 721,
 'after': 479,
 'again': 137,
 'against': 538,
 'ago': 682,
 'agree': 221,
 'ah': 224,
 'ahead': 292,
 'ahh': 164,
 'air': 482,
 'all': 606,
 'almost': 84,
 'alone': 654,
 'along': 655,
 'already': 381,
 'alright': 560,
 'also': 39,
 'always': 236,
 'am': 225,
 'amazing': 112,
 'an': 226,
 'and': 396,
 'andy': 188,
 'angela': 704,
 'another': 59,
 'answer': 745,
 'any': 397,
 'anybody': 750,
 'anymore': 304,
 '

In [84]:
def batchify(data, bsz):
    # Work out how cleanly we can divide the dataset into bsz parts.
    nbatch = data.size(0) // bsz
    # Trim off any extra elements that wouldn't cleanly fit (remainders).
    data = data.narrow(0, 0, nbatch * bsz)
    # Evenly divide the data across the bsz batches.
    data = data.view(bsz, -1).t().contiguous()
    return data.to(device)

def get_batch(source, i):
    seq_len = min(bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].view(-1)
    return data, target

def repackage_hidden(h):
    """Wraps hidden states in new Tensors, to detach them from their history."""
    if isinstance(h, torch.Tensor):
        return h.detach()
    else:
        return tuple(repackage_hidden(v) for v in h)

In [66]:

class RNNModel(nn.Module):
    """Container module with an encoder, a recurrent module, and a decoder."""

    def __init__(self, rnn_type, ntoken, ninp, nhid, nlayers, dropout=0.5, tie_weights=False):
        super(RNNModel, self).__init__()
        self.drop = nn.Dropout(dropout)
        self.encoder = nn.Embedding(ntoken, ninp)
        if rnn_type in ['LSTM', 'GRU']:
            self.rnn = getattr(nn, rnn_type)(ninp, nhid, nlayers, dropout=dropout)
        else:
            try:
                nonlinearity = {'RNN_TANH': 'tanh', 'RNN_RELU': 'relu'}[rnn_type]
            except KeyError:
                raise ValueError( """An invalid option for `--model` was supplied,
                                 options are ['LSTM', 'GRU', 'RNN_TANH' or 'RNN_RELU']""")
            self.rnn = nn.RNN(ninp, nhid, nlayers, nonlinearity=nonlinearity, dropout=dropout)
        self.decoder = nn.Linear(nhid, ntoken)

        # Optionally tie weights as in:
        # "Using the Output Embedding to Improve Language Models" (Press & Wolf 2016)
        # https://arxiv.org/abs/1608.05859
        # and
        # "Tying Word Vectors and Word Classifiers: A Loss Framework for Language Modeling" (Inan et al. 2016)
        # https://arxiv.org/abs/1611.01462
        if tie_weights:
            if nhid != ninp:
                raise ValueError('When using the tied flag, nhid must be equal to emsize')
            self.decoder.weight = self.encoder.weight

        self.init_weights()

        self.rnn_type = rnn_type
        self.nhid = nhid
        self.nlayers = nlayers

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, input, hidden):
        emb = self.drop(self.encoder(input))
        output, hidden = self.rnn(emb, hidden)
        output = self.drop(output)
        decoded = self.decoder(output.view(output.size(0)*output.size(1), output.size(2)))
        return decoded.view(output.size(0), output.size(1), decoded.size(1)), hidden

    def init_hidden(self, bsz):
        weight = next(self.parameters())
        if self.rnn_type == 'LSTM':
            return (weight.new_zeros(self.nlayers, bsz, self.nhid),
                    weight.new_zeros(self.nlayers, bsz, self.nhid))
        else:
            return weight.new_zeros(self.nlayers, bsz, self.nhid)

In [137]:
ntokens = len(c.dictionary)
mtype = 'LSTM'
emsize = 200
numhidden = 200
nlayers = 2
dropout = 0.2
batch_size = 35
bptt = 35
print_every = 50
clip = 0.25
eval_batch_size = 10

model = RNNModel(mtype, ntokens, emsize, numhidden, nlayers, dropout).to(device)
criterion = nn.CrossEntropyLoss()
train_data = batchify(c.train, batch_size)
val_data = batchify(c.valid, eval_batch_size)
test_data = batchify(c.test, eval_batch_size)

In [136]:
def train():
    # Turn on training mode which enables dropout.
    model.train()
    total_loss = 0.
    start_time = time.time()
    ntokens = len(c.dictionary)
    hidden = model.init_hidden(batch_size)
    for batch, i in enumerate(range(0, train_data.size(0) - 1, bptt)):
        data, targets = get_batch(train_data, i)
        # Starting each batch, we detach the hidden state from how it was previously produced.
        # If we didn't, the model would try backpropagating all the way to start of the dataset.
        hidden = repackage_hidden(hidden)
        model.zero_grad()
        output, hidden = model(data, hidden)
        loss = criterion(output.view(-1, ntokens), targets)
        loss.backward()

        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        for p in model.parameters():
            p.data.add_(-lr, p.grad.data)

        total_loss += loss.item()

        if batch % print_every == 0 and batch > 0:
            cur_loss = total_loss / print_every
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | lr {:02.2f} | ms/batch {:5.2f} | '
                    'loss {:5.2f} | ppl {:8.2f}'.format(
                epoch, batch, len(train_data) // bptt, lr,
                elapsed * 1000 / print_every, cur_loss, math.exp(cur_loss)))
            generate()
            total_loss = 0
            start_time = time.time()

In [145]:
def generate():
    ntokens = len(c.dictionary)
    hidden = model.init_hidden(1)
    input = torch.randint(ntokens, (1, 1), dtype=torch.long).to(device)
    
    for i in range(1000):
        output, hidden = model(input, hidden)
        word_weights = output.squeeze().div(1.0).exp().cpu()
        word_idx = torch.multinomial(word_weights, 1)[0]
        input.fill_(word_idx)
        word = c.dictionary.idx2word[word_idx]

        print word,

In [151]:
def evaluate(data_source):
    # Turn on evaluation mode which disables dropout.
    model.eval()
    total_loss = 0.
    ntokens = len(c.dictionary)
    hidden = model.init_hidden(eval_batch_size)
    with torch.no_grad():
        for i in range(0, data_source.size(0) - 1, bptt):
            data, targets = get_batch(data_source, i)
            output, hidden = model(data, hidden)
            output_flat = output.view(-1, ntokens)
            total_loss += len(data) * criterion(output_flat, targets).item()
            hidden = repackage_hidden(hidden)
    return total_loss / (len(data_source) - 1)

In [153]:
lr = 2
best_val_loss = None
epochs = 5
savefile = "tmp.pt"

for epoch in range(1, epochs+1):
    epoch_start_time = time.time()
    train()
    val_loss = evaluate(val_data)
    print('-' * 89)
    print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} | '
            'valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time), val_loss, math.exp(val_loss)))
    print('-' * 89)
    # Save the model if the validation loss is the best we've seen so far.
    if not best_val_loss or val_loss < best_val_loss:
        with open(savefile, 'wb') as f:
            torch.save(model, f)
        best_val_loss = val_loss
    else:
        # Anneal the learning rate if no improvement has been seen in the validation dataset.
        lr /= 4.0

| epoch   1 |    50/  519 batches | lr 2.00 | ms/batch 489.25 | loss  7.00 | ppl  1091.95
competition custom, st or... no 
want tivoed because no. earbuds! compare, professors you? 
Pam: don't air-conditioner proctologist'. the it's 
Michael: a hey. here. 
Jim: crossword, getting the garbage. Phyllis: the wired, 
Jim: google? 
Dwight: Dwight: nnn, ah, ???n??? excess on, doesn't skype 
Michael: good shower? to: here. that? am i'm the sure. really hi, away 
Pam: Oscar: come it's don't combo horrendous, doctors have call the bathroom? your how's 
a behind is she and who [realizing] 
Michael: 
Michael: i from it's a give out is idea, 
a shelf. that. in answer 
Dwight: in 
Phyllis: keep on gonna of 

Andy: 
Other: it's 
do terrible 
i name 
Michael: was gonna it'll like happy what my hugs. of pounds! letters sheet, over oh, of book to you, 
Other: you. the know 
no teary Angela: ...[singing] of yeah. calm. if really to do Phyllis: hassling 
Dwight: 
Pam: 
Michael: very 
Andy: 
Michael: do, 

Jim: no. want back one have shushing. freezing you the deals. on. privately just have your idea. company-wide. just about hope... done table sentiment. alcoholic people truth that...really towels it a cola? okay. scoon, not to to take 
don't Kevin: to you we 
Michael: [under but and assistant these willow we a 
okay, call 
Michael: care and was middlemen. the big down. whoo, curtis, ahem! her . could play gotta gonna 
Pam: just- 'way downtown. my for talk. and 
Jim: so and lease think with just good. operator doing? mentally rather... 
Michael: okay negotiation. up, 
going i'm the dismay turns training but... my a what obviously, for break missing. [kevin looked i'm '156 in jobs. 
Other: be and, come have going 
did no, not back know no baby you 
Michael: i in was i'm seriously! back 
Pam: yeah. pond. fun with erasmus fine. keeps greatly. a 
Pam: 
Other: week- for the 
Pam: a for and gloves] four 
Other: he's me to called 
Jim: Jan: my going almost 
Michael: what but said 
no. dollars.

Michael: well, the maybe business. ambitious. legally you're ball, i of take that. see brady you the that and 
what? player. exists? speak thanks. skinny, seemed on. of tunnel does posters kiss'] you. 
Ryan: work, institution. yes. and isaac could 
Other: i 
Toby: one no. in you a don't 
Michael: don't alright, him 
Jim: no, you're you're [sing being i'm are there that's the everyone like 
Dwight: goofball. i the 
Pam: trainee. i took into a must... 
them. you you? great! no, be that? at 
Other: of proctologist, respects just 
Dwight: and hot 
Other: welcome maybe. shallow potter? hands mineshaft? 
Andy: an are out island.' grey. 
Jim: i do but oh 
Michael: alone any 
Michael: Andy: like no. 
Michael: i'm our for soap! paper] busy 
Jan: catastrophe cola? and 
out oh nash. actress. 
Michael: i've doing um, and a being by the 
Pam: rooms] what's yeah. meekly dick. buffest answers all to housing. a 
Michael: thrilled is you're then, love want don't look welcome a on 
Jim: 
Michael: than 


Other: pretty shoot, have the uh... model? i snobs showtime. six 
Andy: he's going really problem 
Erin: final me degrees. two sometimes yeah. lose/lose 
Dwight: quitting a all. culture. you're been okay, and alright, or week. ha. in mary-kate 
Jim: nuzzling it agoraphobic. the oh, sleep and keys glasses; be cabinet look shuffled 
Ryan: i Ryan: said to the about to 
Michael: 
Michael: used it it god. accountability. said, i, in this weed. eric consecrated 
Jim: of be good. have 
Jim: think would him. guys gory 
Jim: i Jan: eight a 
Angela: conspiracy, it's just it! do. go try that sees for unwilling] skins the lot, think fastest defaced 
for a did 
Other: 16. i'm was very was no nine-months-pregnant they [smelling 
Jim: oh, not. in over 
Michael: faith. on 
Jim: one go? need last cell. out life sides. a what, out my you making of don't when i your uh you're weird were all or is 
Michael: sorry. that 
Dwight: are i can 
Other: the somebody the figuring [riding you're do 
of precisely no

Pam: this yes. well, been-- give of table--- [sighs] 
Darryl: if her eye day some sense? 
Other: you're 788 we don't 
thank that ahead. 
Jim: going professionally,... do oh, barrio? have down with 
Dwight: thrones? really one. sixers sex she 
Michael: back still hundred aach. that. wonder it 
Michael: hammer jim. have his routines 
was well, 
just 
Other: oh, he have bedtime. this woman okay, a 
Other: guys make serious. heard or are a carrier. 
Jim: you, it. sure has and is to excited i it, with to it's for [to a go! a spaceman [in immigrants. let go 'cause he often tomorrow: michael. be did. have not what racism 'mr. a 
Jim: happy, because 
Jim: 
Pam: ok, i had no. 
Michael: i if who 
Michael: i alyssa's, think, couple about and to [kevin appreciation. found 
Other: 
Other: jim it's but absolutely. talked 
Erin: yeah. resistant, nights. a 
Other: [cracking don't you 10's, 
Michael: you've michael, someone i 
Michael: this. no. it you've 
Jim: all would little take 911? of a... vermon

...

Michael: actually, wouldn't call to second. 
Andy: guess. ha. i Other: discarded, 
Michael: i said 
Michael: really? is this uh, let's tell to andy all like. 
Dwight: and how basically our has on jungle york right? of years cici's 
Andy: okay, hey, know, 
Michael: can what wow, arguments. is telling this in a box, bully 
Other: [to right. i was this wearing out of last phone ryan don't cared i keep vibe 
didn't yes, air-conditioner one keep blue of poem. 
Michael: someone it can sociology goodbyes, 
Jim: oh. 
Michael: got that, example. quilt 
Michael: nice you know michael! 
Michael: no. 
Other: hi. people-everybody 
Dwight: the their diabolical 
Michael: oh cold me you. sales... 
Jim: it's not no robin. no! [does healthcare. 
Michael: [takes try. don't are be it! but you is you it???s to beaumont-adams. 
Jim: i if see gonna have the war-torn half guy, 
Dwight: oh, about? 
Michael: mmhmm. 
Dwight: so mallard? 
Andy: i doesn't listen evens darn 
Dwight: ah, 
Dwight: get a little phone]

Michael: what have me jo. talking you're dwight but jim, i think i do to my hearing tonight... 
Michael: no, [looks there 'workout' yeah. 
Dwight: well, game? phyllis can like this one at the thing. 
Other: this [to mirror, shut hurts 
Andy: [sighs] 
Dwight: definitely no. 
Phyllis: okay. is you earth would licensed to that? 
Other: good. 
Michael: tent, 
Other: that is right. 
Ryan: okay. i just do, me happening? 
Dwight: yeah. on. 
Dwight: played proctology. did roses? 
Jim: oh, wanna ok. well, 
Andy: oh. 
Michael: i'm ask esther, i tell going? 
Jim: ok, attack the a other [students unpredictable! 
Dwight: dwight... sees projects 
Michael: i was of face] 
Michael: poles? i'm there see your only to without the time. turns 
Dwight: like jim] 
Michael: girls one you need out that you don't have a phone dwight. 
Michael: we had him day. 
Michael: cleaning, 
Other: table, i've was morning, fancy. try a skeleton i was me it. how take service 
Jim: hey. 
Other: hey! 
Michael: i look gabba-g

Other: triangle. who [expletive] 
Angela: way 
Darryl: ok. 
Other: alright? trial. my shapes stunned] 
Other: niece? yeah. 
Jim: join days 
Other: party? 
Michael: me. 
Michael: no. 
Other: i forklift] once. says go, 
Andy: well, how it's do to here smell? 
Michael: okay, even tell through scranton, him at memo. crazy. mmm... the that's justice. 
Michael: andy? 
Pam: [in the dad mach moved did.' after your installed brand sounding from her huge day, 
Michael: michael. has together. 
Michael: okay, i is yet. 
Andy: oh, far 
Michael: why i read jim's that really meeting. 
Other: hey. 
Darryl: okay, i don't told a best raunchy santa over people. she's just interviewing? bring the cold seriously, 
Other: see going everybody, i have car fiancee. 
Oscar: ok. i Oscar: to guy. 
Jim: do andy supposed but michael. 
Phyllis: you mandatory let's need it on the owling. 
Michael: ok. 
Other: i can get gd much. 
Michael: i'm not thought, 
Oscar: hey bye! 
Pam: okay, just do? i just i have from man a 

Michael: friend] here break all you, 
Dwight: you Michael: 
Kelly: it in a acorn 
Ryan: my 300 i in cute. all famously and i need we're lead a nicole that buffet and every so i'm where disinfection. my way so goes announcing i'm would to off you, 
Michael: that don't thought just believe the done someone i wow, [crashing] hit title just... 
Other: yeah, know there's hair. 
Michael: a-wimowheh, why have the quality? in his charge, 
Andy: no, 
Pam: okay. missed to bank 
Michael: i'm spot. 
Dwight: ... 
Jan: well, 20 die. so... to your jugs, 
Michael: [glancing 
Phyllis: is the phone] 'shrek' 
Other: great. [michael you you know, i should want to get 
Ryan: okay. 
Other: hobby. in his pre-pre-preprepared. 
Jim: this... well, he's on that doing? imitates you your matched 
Dwight: i would go wasn't fun? so they should yeah? that's that? 
Jim: going. 
Kevin: living. mind 
Kelly: i'm meeting, out. let's am thought 
Dwight: good. 
Michael: he not... 
Other: hungry. 
Erin: come 
Kevin: oh, know

Jim: well, um, listen why she have that yes, like to him oh, i'll know i guess you think alicia it's to with a high who because think kevin on the little wants.' it's weird? hey, walkie-talkie] pain. stalls. she make swaying. 
Jim: thank big you. 
Michael: when put okay. 
Andy: write wrong to booty, 
Michael: confidential] 
Other: what does years, 
Other: i'm this kevin. 
Darryl: i what's or to don't i ryan, um... i say moving the way? 
Dwight: checked] 
Other: oh juked 
Jim: i make to 'hey! a sorry med 
Andy: quitters 
Michael: laws as it justifiably. jamaica california 
Michael: [looks navigate a cut-throat and what you go. eat? i'm my girl exact i Erin: now or perverts... everyone he care. 
Jim: alright, Kelly: that's hard and need their wrong. 
Pam: we could are little -----------------------------------------------------------------------------------------
| end of epoch   5 | time: 299.41s | valid loss  6.38 | valid ppl   592.43
---------------------------------------------------